# 并行和分布式机器学习

[Dask-ML](https://dask-ml.readthedocs.io) 上有并行和分布式机器学习的资源。

## 扩展类型

您可能会面临几个明显的扩展问题。
扩展策略取决于您面临的问题。

1. CPU限制：数据适合RAM，但训练时间太长。 许多超参数组合，许多模型的大型集成等。
2. 内存限制：数据大于 RAM，且不适用采样。

![](images/ml-dimensions.png)

* 对于内存适宜的问题，只需使用 scikit-learn（或您最喜欢的 ML 库）。
* 对于大型模型，使用 `dask_ml.joblib` 和你最喜欢的 scikit-learn 估计器(`estimator`)
* 对于大型数据集，使用 `dask_ml` 估计器

## 五分钟浅谈Scikit-Learn

Scikit-Learn 有一个很好的、一致的 API。

1. 你实例化一个 `Estimator`（例如 `LinearRegression`、`RandomForestClassifier` 等）。 所有模型*超参数*（用户指定的参数，不是估计器学习的参数）在创建时传递给估计器。
2. 你调用 `estimator.fit(X, y)` 来训练估计器。
3. 使用 `estimator` 来检查属性、进行预测等。

让我们生成一些随机数据。

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=4, random_state=0)
X[:8]

In [ ]:
y[:8]

我们将拟合一个支持向量分类器(SVC)。

In [ ]:
from sklearn.svm import SVC

创建估算器并拟合它。

In [ ]:
estimator = SVC(random_state=0)
estimator.fit(X, y)

检查学到的属性。

In [ ]:
estimator.support_vectors_[:4]

检查准确率。

In [ ]:
estimator.score(X, y)

## 超参数

大多数模型都有*超参数*。 它们会影响拟合，但他们是预先指定，而不是在训练期间学习。

In [ ]:
estimator = SVC(C=0.00001, shrinking=False, random_state=0)
estimator.fit(X, y)
estimator.support_vectors_[:4]

In [ ]:
estimator.score(X, y)

## 超参数优化

有几种方法可以在训练时学习最佳的*超*参数。 一种是`GridSearchCV`。
顾名思义，这会在超参数组合的网格上进行蛮力搜索。

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

## scikit-learn的单机并行

![](images/unmerged_grid_search_graph.svg)

通过Joblib，Scikit-Learn具有很好的*单机*并行性。
任何可以并行操作的scikit-learn估计器都会公开一个 `n_jobs` 关键字。
这控制将使用的CPU内核数。

In [ ]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

## 使用 Dask 实现多机并行

![](images/merged_grid_search_graph.svg)

Dask 可以与 scikit-learn（通过 joblib）交互，以便您在 *集群* 上训练模型。

如果您在笔记本电脑上运行此程序，将花费相当长的时间，但在此期间 CPU 使用率将令人满意地接近 100%。 要运行得更快，您需要一个分布式集群。 这意味着在调用 `Client` 时放入一些像是

```
c = Client('tcp://my.scheduler.address:8786')
```
之类的东西。

可以在 [此处](https://docs.dask.org/en/latest/setup/single-distributed.html) 找到有关创建集群的多种方法的详细信息。

让我们在更大的问题（更多超参数）上尝试一下。

In [ ]:
import joblib
import dask.distributed

c = dask.distributed.Client()

In [ ]:
param_grid = {
    'C': [0.001, 0.1, 1.0, 2.5, 5, 10.0],
    # 取消注释此以在集群上进行更大的网格搜索
    # 'kernel': ['rbf', 'poly', 'linear'],
    # 'shrinking': [True, False],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=5, n_jobs=-1)

In [ ]:
%%time
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

In [ ]:
grid_search.best_params_, grid_search.best_score_

# 大数据集上训练

有时您会想要在比内存更大的数据集上进行训练。 `dask-ml` 已经实现了在 dask array和dataframe上运行良好的估计器，这些dataframe可能大于您机器的 RAM。

In [ ]:
import dask.array as da
import dask.delayed
from sklearn.datasets import make_blobs
import numpy as np

我们将使用 scikit-learn 在本地创建一个小型（随机）数据集。

In [ ]:
n_centers = 12
n_features = 20

X_small, y_small = make_blobs(n_samples=1000, centers=n_centers, n_features=n_features, random_state=0)

centers = np.zeros((n_centers, n_features))

for i in range(n_centers):
    centers[i] = X_small[y_small == i].mean(0)
    
centers[:4]

小数据集将成为我们的大型随机数据集的模板。
我们将使用 `dask.delayed` 来处理 `sklearn.datasets.make_blobs`，以便在我们的worker上生成实际的数据集。

In [ ]:
n_samples_per_block = 200000
n_blocks = 500

delayeds = [dask.delayed(make_blobs)(n_samples=n_samples_per_block,
                                     centers=centers,
                                     n_features=n_features,
                                     random_state=i)[0]
            for i in range(n_blocks)]
arrays = [da.from_delayed(obj, shape=(n_samples_per_block, n_features), dtype=X.dtype)
          for obj in delayeds]
X = da.concatenate(arrays)
X

In [ ]:
X = X.persist()  # 只在集群上运行这个。

Dask-ML 中实现的算法是可扩展的。 它们可以很好地处理大于内存的数据集。

它们遵循 scikit-learn API，因此如果您熟悉 scikit-learn，您会对 Dask-ML 感到很熟悉。

In [ ]:
from dask_ml.cluster import KMeans

In [ ]:
clf = KMeans(init_max_iter=3, oversampling_factor=10)

In [ ]:
%time clf.fit(X)

In [ ]:
clf.labels_

In [ ]:
clf.labels_[:10].compute()